# Newton-Raphson: Python to IR
Through the course of this notebook, we will translate an implementation of Newton-Raphson iteration written in Python into its IR. First, let's install the packages we'll need to write the code:

In [ ]:

from scipy import misc

In [ ]:
# Newton Raphson Method for extended example of creation IR:
# The function is : (4/3)x^3 - (3/2)x^2 + 8
def function( x ):
    return 4/3*x * x * x - 3/2*x * x + 8
 
# Derivative of the function
def derivedFunction( x ):
    return 4 * x * x - 3 * x
 
# Function to find the root
def newtonIteration( x ):
    h = function(x) / derivedFunction(x)
    while abs(h) >= 0.0001:
        h = function(x)/derivedFunction(x)
         
        # x(i+1) = x(i) - f(x) / f'(x)
        x = x - h
        print("Iterations =" ,x)
    print("The value of the root is : ",
                             "%.4f"% x)

# Testing
x0 = 5 # Initial value
newtonIteration(x0)
# As a results, we'd like to find -1.5092

Now that we can see it's working, it's time to look at the lower level representations of our code. Since it's Python, we have the added ability of looking at its bytecode using `dis`, and of course we'll be interested in the AST as well.

In [2]:
import dis
import ast

To view the bytecode, we disassemble the main driver function.

In [3]:
dis.dis(newtonIteration)

 12           0 LOAD_GLOBAL              0 (function)
              2 LOAD_FAST                0 (x)
              4 CALL_FUNCTION            1
              6 LOAD_GLOBAL              1 (derivedFunction)
              8 LOAD_FAST                0 (x)
             10 CALL_FUNCTION            1
             12 BINARY_TRUE_DIVIDE
             14 STORE_FAST               1 (h)

 13     >>   16 LOAD_GLOBAL              2 (abs)
             18 LOAD_FAST                1 (h)
             20 CALL_FUNCTION            1
             22 LOAD_CONST               1 (0.0001)
             24 COMPARE_OP               5 (>=)
             26 POP_JUMP_IF_FALSE       64

 14          28 LOAD_GLOBAL              0 (function)
             30 LOAD_FAST                0 (x)
             32 CALL_FUNCTION            1
             34 LOAD_GLOBAL              1 (derivedFunction)
             36 LOAD_FAST                0 (x)
             38 CALL_FUNCTION            1
             40 BINARY_TRUE_DIVIDE
         

The new elements of this example - in particular, function calls - are handled very elegantly in the bytecode. The virtual machine which runs the bytecode for python maintains several stack data structures, so the instruction `CALL_FUNCTION` is as easy as popping the parameters and function name off of the correct stack. The looping is similarly implemented by a pair of `JUMP` instructions, one conditional (instruction 26) and one absolute (not visible in the preview, but instruction 62 in the full output data) which sends us back to instruction 16.

Next, the ast package makes it easy for us to view the AST for our code:

In [ ]:
print(ast.dump(ast.parse("""
# Newton Raphson Method for extended example of creation IR:
# The function is : (4/3)x^3 - (3/2)x^2 + 8
def function( x ):
    return 4/3*x * x * x - 3/2*x * x + 8
 
# Derivative of the function
def deriveFunction( x ):
    return 4 * x * x - 3 * x
 
# Function to find the root
def newtonIteration( x ):
    h = function(x) / deriveFunction(x)
    while abs(h) >= 0.0001:
        h = function(x)/deriveFunction(x)
         
        # x(i+1) = x(i) - f(x) / f'(x)
        x = x - h
        print("Iterations =" ,x)
    print("The value of the root is : ",
                             "%.4f"% x)

# Testing
x0 = 5 # Initial value
newtonIteration(x0)
# As a results we need to find -1.5092
"""), indent=4))

It's not terribly instructive, but gives a good idea of the internal structure which can be imposed on the code. The [Numba](Newton_Raphson.rst) and [LLVM IR](Newton_Raphson.ll) are not very instructive either, but we link to them for the sake of completeness.